# ------------ TIM Python Client - System Driven Anomaly Detection ------------

# 0. Setup

In [3]:
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as splt

In [3]:
import tim
client = tim.Tim(email='',password='',server='')

In [5]:
def call_tim_function(action,object_type,arguments=None):
    functions = {
        'list':{
            'user_group':client.user_groups.list_user_group,
            'workspace':client.workspaces.list_workspace,
            'dataset':client.datasets.list_dataset,
            'dataset_version':client.datasets.list_dataset_versions,
            'use_case':client.use_cases.list_use_case,
            'experiment':client.experiments.list_experiment
        },
        'create':{
            'user_group':client.user_groups.create_user_group,
            'workspace':client.workspaces.create_workspace,
            'use_case':client.use_cases.create_use_case,
            'experiment':client.experiments.create_experiment
        }
    }
    response = functions[action][object_type](**arguments)
    return response

def create_tim_object_configuration(pipeline,object_type,parameters,name):
    
    if object_type=='user_group':
        configuration = {"name": name,"users": [{"id": client.users.details_user()['id'],"isOwner": True}]}
    if object_type=='workspace':
        configuration = {"name": name,"userGroup": {"id": parameters['user_group_id']}}
    if object_type=='dataset':
        try:
            versionName = pipeline['dataset_version']['name']
        except:
            versionName = 'initial upload'
        configuration = {"name": name,"workspace": {"id": parameters['workspace_id']},"versionName":versionName}
    if object_type=='dataset_version':
        configuration = update_dataset_configuration = {"versionName": name}
    if object_type=='use_case':
        configuration = {"name": name,"workspace": {"id": parameters['workspace_id']},"dataset": {"id": parameters['dataset_id']}}
    if object_type=='experiment':
        configuration = {"name": name,"useCase": {"id": parameters['use_case_id']},"type": pipeline['experiment']['create']['type']}
    return configuration

def check_tim_object(pipeline,object_type,parameters):
    try:
        object_id = pipeline[object_type]['id']
        print(object_type,'id available.')
    except:
        try:
            object_name = pipeline[object_type]['name']
            object_list = [f for f in call_tim_function('list',object_type,parameters) if f['name']==object_name]
            tim_object = object_list[0]
            object_id = tim_object['id']
            print(object_type,'found by name.')
        except:
            try:
                add_to_configuration = pipeline[object_type]['create']['configuration']
                object_name = add_to_configuration['versionName'] if object_type == 'dataset_version' else add_to_configuration['name']
                create_configuration = create_tim_object_configuration(pipeline,object_type,parameters,object_name)
                object_configuration = {**add_to_configuration, **create_configuration}
            except:
                object_name = pipeline[object_type]['name'] 
                object_configuration = create_tim_object_configuration(pipeline,object_type,parameters,object_name)
            if object_type == 'dataset':
                tim_file = pipeline[object_type]['create']['file']
                tim_upload = client.upload_dataset(
                    dataset = tim_file,
                    configuration = object_configuration,
                    outputs = ['response'],
                    status_poll = print,
                    tries_left = 300
                )
                tim_object = tim_upload.response
            elif object_type == 'dataset_version':
                tim_file = pipeline[object_type]['create']['file']
                tim_update = client.update_dataset(
                    dataset_id = parameters['id'],
                    dataset_version = tim_file,
                    configuration = object_configuration,
                    outputs = ['response'],
                    status_poll = print,
                    tries_left = 300
                )
                tim_object = tim_update.response['version']                
            else:
                tim_object = call_tim_function('create',object_type,{'configuration':object_configuration})
            object_id = tim_object['id']
            print(object_type,'created.')
    return object_id

def tim_pipeline_setup(pipeline):
    try:
        response = {'name':pipeline['name']}
    except:
        pass
    try:
        user_group_id = check_tim_object(pipeline=pipeline,object_type='user_group',parameters={})
        response['user_group'] = user_group_id
    except:
        pass
    try:
        workspace_id = check_tim_object(pipeline=pipeline,object_type='workspace',parameters={'user_group_id':user_group_id})
        response['workspace'] = workspace_id
    except:
        pass
    dataset_id = check_tim_object(pipeline=pipeline,object_type='dataset',parameters={'workspace_id':workspace_id})
    response['dataset'] = dataset_id
    try:
        dataset_version_id = check_tim_object(pipeline=pipeline,object_type='dataset_version',parameters={'id':dataset_id})
        response['dataset_version'] = dataset_version_id
    except:
        pass
    try:
        use_case_id = check_tim_object(pipeline=pipeline,object_type='use_case',parameters={'workspace_id':workspace_id,'dataset_id':dataset_id})
        response['use_case'] = use_case_id    
    except:
        pass
    try:
        experiment_id = check_tim_object(pipeline=pipeline,object_type='experiment',parameters={'use_case_id':use_case_id})
        response['experiment'] = experiment_id   
    except:
        pass
    return response

# 1. Data Preparation

In [6]:
source_df = pd.read_parquet('dataset.parquet')
print(source_df.columns)

Index(['timestamp', 'BROKEN', 'M_casing_vib', 'M_freq_A', 'M_freq_B',
       'M_freq_C', 'M_speed', 'M_current', 'M_active_power',
       'M_apparent_power', 'M_reactive_power', 'M_shaft_power',
       'M_phase_current_A', 'M_phase_current_B', 'M_phase_current_C',
       'M_Coupling Vibration', 'M_Phase Voltage AB', 'M_Phase Voltage BC',
       'M_Phase Voltage CA', 'P_Casing Vibration', 'P_stg_1_imp_speed_A',
       'P_stg_1_imp_speed_B', 'P_stg_1_imp_speed_C', 'P_stg_1_imp_speed_D',
       'P_stg_1_imp_speed_E', 'P_stg_1_imp_speed_F', 'P_stg_2_imp_speed_A',
       'P_stg_2_imp_speed_B', 'P_stg_2_imp_speed_C', 'P_stg_2_imp_speed_D',
       'P_stg_2_imp_speed_E', 'P_stg_2_imp_speed_F', 'P_stg_2_imp_speed_G',
       'P_stg_2_imp_speed_H', 'P_stg_2_imp_speed_I', 'P_inlet_flow',
       'P_discharge_flow', 'P_UNKNOWN', 'P_lube_oil_reservoir',
       'P_lube_oil_return_temp', 'P_lube_oil_supply_temp',
       'P_thrust_act_bearing_temp', 'M_bearing_temp_1', 'M_bearing_temp_2',
       'P_thru

In [7]:
tim_dataset = source_df.copy()
timestamp = 'timestamp'
labels = ['BROKEN','NORMAL','RECOVERING']
target = labels[0]
predictors = [s for s in list(tim_dataset.columns) if s not in [timestamp]+labels]
tim_dataset[timestamp] = pd.to_datetime(tim_dataset[timestamp],format='%Y-%m-%dT%H:%M:%SZ')
tim_dataset

,timestamp,BROKEN,M_casing_vib,M_freq_A,M_freq_B,M_freq_C,M_speed,M_current,M_active_power,M_apparent_power,...,P_thrust_inct_bearing_temp,P_drive_bearing_temp_1,P_bearing_temp_1,P_bearing_temp_2,P_drive_bearing_temp_2,P_inlet_pressure,P_inlet_temp,P_discharge_pressure_2,NORMAL,RECOVERING
0,2018-04-01 00:00:00,0,2.46539,47.0920,53.2118,46.3108,634.375,76.4597,13.4115,16.1314,...,41.9271,39.6412,65.6829,50.9259,38.1944,157.986,67.7083,201.389,1,0
1,2018-04-01 00:01:00,0,2.46539,47.0920,53.2118,46.3108,634.375,76.4597,13.4115,16.1314,...,41.9271,39.6412,65.6829,50.9259,38.1944,157.986,67.7083,201.389,1,0
2,2018-04-01 00:02:00,0,2.44473,47.3524,53.2118,46.3976,638.889,73.5460,13.3247,16.0373,...,41.6667,39.3519,65.3935,51.2153,38.1944,155.961,67.1296,203.704,1,0
3,2018-04-01 00:03:00,0,2.46047,47.0920,53.1684,46.3976,628.125,76.9890,13.3174,16.2471,...,40.8854,39.0625,64.8148,51.2153,38.1944,155.961,66.8403,203.125,1,0
4,2018-04-01 00:04:00,0,2.44572,47.1354,53.2118,46.3976,636.458,76.5890,13.3536,16.2109,...,41.4062,38.7732,65.1042,51.7940,38.7732,158.275,66.5509,201.389,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220315,2018-08-31 23:55:00,0,2.40735,47.6996,50.5208,43.1424,634.722,64.5910,15.1186,16.6522,...,38.2812,68.2870,52.3727,48.3218,41.0880,212.384,153.6460,231.192,1,0
220316,2018-08-31 23:56:00,0,2.40046,47.6996,50.5642,43.1424,630.903,65.8336,15.1548,16.7028,...,38.2812,66.8403,50.6366,48.0324,40.7986,213.831,156.2500,231.192,1,0
220317,2018-08-31 23:57:00,0,2.39653,47.6996,50.5208,43.1424,625.926,67.2944,15.0897,16.7028,...,39.0625,65.3935,48.9005,48.0324,40.7986,217.303,155.3820,232.060,1,0
220318,2018-08-31 23:58:00,0,2.40637,47.6996,50.5208,43.1424,635.648,65.0918,15.1186,16.5654,...,40.6250,64.2361,47.7431,48.3218,40.5093,222.512,153.9350,234.086,1,0


# 2. TIM

## 2.1 Workflow

In [8]:
upload_dataset_configuration = {
    # "timestampColumn": timestamp,
    # "groupKeys": group_keys,
    "name": "pump_sensor"
}
# -----------------------------------------------------------------------------------------------------------------------
user_group = {
    # 'id':'135c50cd-e6ea-423e-ae2b-581564cb9cbc',
    'name':'POV',
    # 'create':{'configuration':create_user_group_configuration}
}
workspace = {
    # 'id':'e3e34c8f-3864-4199-af4b-70366c6a79db',
    'name':'Templates',
    # 'create':{'configuration':create_workspace_configuration}
}
dataset = {
    # 'id':'0c217702-6d7c-4349-9345-a5d8c9120881',
    # 'name':upload_dataset_configuration['name'],
    'create':{'configuration':upload_dataset_configuration,'file':tim_dataset}
}
dataset_version = {
    # 'id':'03ea3953-3956-4155-8ad8-f3c95daadd5f',
    # 'name':'panel_data_demo',
    # 'create':{'configuration':update_dataset_configuration,'file':tim_dataset.tail(28)}
}
use_case = {
    # 'id':'3b8ad8ce-8516-4ed4-a9e7-e891ed5e176c',
    'name':'pump sensor',
    # 'create':{'configuration':create_use_case_configuration}
}
experiment = {
    # 'id':'bc82c706-be70-4726-8976-dbadf75e7385',
    'name':'System Driven Anomaly Detection',
    'create':{
        # 'configuration':create_use_case_configuration,
        'type':'AnomalyDetection'
    }
}
# -----------------------------------------------------------------------------------------------------------------------
pipeline_input = {
    'name':'pipeline_1',
    'user_group':user_group,
    'workspace':workspace,
    'dataset':dataset,
    'dataset_version':dataset_version,
    'use_case':use_case,
    'experiment':experiment
}
# -----------------------------------------------------------------------------------------------------------------------
pipeline_response = tim_pipeline_setup(pipeline=pipeline_input)
pipeline_response

user_group found by name.
workspace found by name.
{'status': 'Queued', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 0.0, 'createdAt': '2023-10-18T11:26:18.164Z'}
{'status': 'Running', 'progress': 20.0, 'createdAt': '2023-10-18T11:26

{'name': 'pipeline_1',
 'user_group': '4f2bbd5d-4b40-473f-9cae-7c03508ae11c',
 'workspace': '7f5a354a-026e-431c-85e9-261ec543329c',
 'dataset': '081edaf0-5af9-4fcf-8008-bfc1758b542e',
 'use_case': 'af411cdd-7120-4622-870a-7e0790f2ae4a',
 'experiment': 'edc99d69-cd1c-4e6c-9f99-461c406e1897'}

In [9]:
dataset_id = pipeline_response['dataset']
use_case_id = pipeline_response['use_case']
experiment_id = pipeline_response['experiment']

## 2.2 Configuration

In [10]:
detection_build_system_model_configuration = {
    "name": "system_driven_default",
    "useCase": {"id": use_case_id},
    "experiment": {"id": experiment_id},
    # "configuration": {
        # "domainSpecifics": {
            # "sensitivity": 1,
            # "minSensitivity": 0,
            # "maxSensitivity": 0,
            # "anomalyIndicatorWindow": {"baseUnit": "Sample","value": 1}
        # },
        # "model": {
        #     "numberOfTrees": 150,
        #     "subSampleSize": 256,
        #     "maxTreeDepth": 7,
        #     "extensionLevel": 1,
        #     "normalization": True
        # }
    # },
    "data": {
        # "version": {"id": "a74ae716-a86e-47f0-8a50-d8b21d6d7dd6"},
        "rows": {"type":"First","baseUnit": "Sample","value": 20000}, # or [{"from": "yyyy-mm-dd HH:MM:SS","to": "yyyy-mm-dd HH:MM:SS"}]
        "columns": predictors+[target],
        "labelColumn": target,
        # "imputation": {"type": "LOCF","maxGapLength": 6},
        # "timeScale": {"baseUnit": "Hour","value": 1}
    }
}
detection_build_system_model_configuration

{'name': 'system_driven_default',
 'useCase': {'id': 'af411cdd-7120-4622-870a-7e0790f2ae4a'},
 'experiment': {'id': 'edc99d69-cd1c-4e6c-9f99-461c406e1897'},
 'data': {'rows': {'type': 'First', 'baseUnit': 'Sample', 'value': 20000},
  'columns': ['M_casing_vib',
   'M_freq_A',
   'M_freq_B',
   'M_freq_C',
   'M_speed',
   'M_current',
   'M_active_power',
   'M_apparent_power',
   'M_reactive_power',
   'M_shaft_power',
   'M_phase_current_A',
   'M_phase_current_B',
   'M_phase_current_C',
   'M_Coupling Vibration',
   'M_Phase Voltage AB',
   'M_Phase Voltage BC',
   'M_Phase Voltage CA',
   'P_Casing Vibration',
   'P_stg_1_imp_speed_A',
   'P_stg_1_imp_speed_B',
   'P_stg_1_imp_speed_C',
   'P_stg_1_imp_speed_D',
   'P_stg_1_imp_speed_E',
   'P_stg_1_imp_speed_F',
   'P_stg_2_imp_speed_A',
   'P_stg_2_imp_speed_B',
   'P_stg_2_imp_speed_C',
   'P_stg_2_imp_speed_D',
   'P_stg_2_imp_speed_E',
   'P_stg_2_imp_speed_F',
   'P_stg_2_imp_speed_G',
   'P_stg_2_imp_speed_H',
   'P_stg_2_i

## 2.3 Model Building

In [11]:
# -----------------------------------------------------------------------------------------------------
detection_build_system_model = client.extended_detection.build_system_model(
    configuration = detection_build_system_model_configuration,
    outputs = [
        'id',
        'logs',
        'table',
    ],
    status_poll = print
 )
# -----------------------------------------------------------------------------------------------------
detection_build_system_model_id = detection_build_system_model.id
detection_build_system_model_logs = detection_build_system_model.logs
detection_build_system_model_table = detection_build_system_model.table

{'status': 'Running', 'createdAt': '2023-10-18T11:30:00.333Z'}
{'status': 'Running', 'createdAt': '2023-10-18T11:30:00.333Z'}
{'status': 'Running', 'progress': 5.0, 'CPU': 0.69, 'memory': 7461.0, 'createdAt': '2023-10-18T11:30:05.797Z'}
{'status': 'Running', 'progress': 27.5, 'CPU': 0.69, 'memory': 7437.0, 'createdAt': '2023-10-18T11:30:07.594Z'}
{'status': 'Running', 'progress': 100.0, 'CPU': 0.79, 'memory': 7439.0, 'createdAt': '2023-10-18T11:30:11.016Z'}
{'status': 'Running', 'progress': 100.0, 'CPU': 0.79, 'memory': 7439.0, 'createdAt': '2023-10-18T11:30:11.016Z'}
{'status': 'Finished', 'progress': 100.0, 'CPU': 0.79, 'memory': 7439.0, 'createdAt': '2023-10-18T11:30:11.016Z'}


In [12]:
detection_build_system_model_table['timestamp'] = pd.to_datetime(detection_build_system_model_table['timestamp'],format='%Y-%m-%dT%H:%M:%S.%fZ')

# 3. Results

## 3.1 Visual

In [ ]:
v_results = detection_build_system_model_table.copy()
v_data = tim_dataset[(tim_dataset[timestamp]>=v_results['timestamp'].min())&(tim_dataset[timestamp]<=v_results['timestamp'].max())].copy()
fig = splt.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=v_results['timestamp'], y=v_results['anomaly_code'], name='anomaly_code'), row=1, col=1)
fig.add_trace(go.Scatter(x=v_results['timestamp'], y=v_results['anomaly_indicator'], name='anomaly_indicator'), row=2, col=1)
fig.add_trace(go.Scatter(x=v_data[timestamp], y=v_data[target], name=target), row=1, col=1)
fig.add_hline(y=1, line_color="orange", row=2, col=1)
fig.update_layout(height=900, width=1200, title_text="Results visualization")
fig.show()    

## 3.2 Anomaly List

In [14]:
anomaly_list_df = detection_build_system_model_table[detection_build_system_model_table['anomaly_code']==1].copy().rename(columns={'anomaly_code':'anomaly'})
anomaly_list_df.sort_values(by='anomaly_indicator',ascending=False)

,timestamp,label,anomaly,anomaly_indicator
17362,2018-04-13 17:08:00,False,1,1.201634
17346,2018-04-13 16:52:00,False,1,1.196700
17354,2018-04-13 17:00:00,False,1,1.190899
17355,2018-04-13 17:01:00,False,1,1.190899
17345,2018-04-13 16:51:00,False,1,1.190699
17359,2018-04-13 17:05:00,False,1,1.189220
17353,2018-04-13 16:59:00,False,1,1.188495
17356,2018-04-13 17:02:00,False,1,1.187291
17358,2018-04-13 17:04:00,False,1,1.186813
17361,2018-04-13 17:07:00,False,1,1.185713


## 3.3 Insights

In [15]:
window_start = 60 # Samples
window_end = 5 # Samples

# anomaly_nb = 0
# anomaly_ts = anomaly_list_df.sort_values(by='anomaly_indicator',ascending=False).iloc[anomaly_nb]['timestamp']
anomaly_ts = tim_dataset[tim_dataset[target]==1][timestamp].iloc[0]
insights_df = tim_dataset.merge(detection_build_system_model_table,on='timestamp',how='left').reset_index(drop=True)
anomaly_index = insights_df[insights_df[timestamp]==anomaly_ts].index[0]
window_df = insights_df.iloc[anomaly_index-window_start:anomaly_index+window_end]
correlation_df = window_df[predictors+['anomaly_indicator']].corr()[['anomaly_indicator']]
correlation_df['sort'] = abs(correlation_df['anomaly_indicator'])

In [ ]:
v_data = correlation_df.sort_values(by='sort',ascending=False)[1:]
fig1 = go.Figure(go.Bar(x=v_data.index, y=v_data['anomaly_indicator']))
fig1.update_layout(height=800,width=1200,title_text='Root Cause Analysis',xaxis_title='name',yaxis_title='rel_importance')
fig1.show()

In [ ]:
fig = splt.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=window_df['timestamp'], y=window_df['anomaly_code'], name='anomaly_code'), row=1, col=1)
fig.add_trace(go.Scatter(x=window_df['timestamp'], y=window_df['anomaly_indicator'], name='anomaly_indicator'), row=2, col=1)
fig.add_hline(y=1, line_color="orange", row=2, col=1)
for idx, p in enumerate(correlation_df.sort_values(by='sort',ascending=False)[1:4].index):
    fig.add_trace(go.Scatter(x=window_df['timestamp'], y=window_df[p], name=p), row=3, col=1)
fig.add_vline(x=anomaly_ts, line_color="red", row=3, col=1)
fig.update_layout(height=900, width=1200, title_text="Top Anomaly Contributor")
fig.show()   

## 2.2 Configuration

In [18]:
subset = list(set(correlation_df[correlation_df['sort']>0.9].index)-set(['anomaly_indicator']))

In [20]:
detection_build_system_model_configuration = {
    "name": "system_driven_subset",
    "useCase": {"id": use_case_id},
    "experiment": {"id": experiment_id},
    # "configuration": {
        # "domainSpecifics": {
            # "sensitivity": 1,
            # "minSensitivity": 0,
            # "maxSensitivity": 0,
            # "anomalyIndicatorWindow": {"baseUnit": "Sample","value": 10}
        # },
        # "model": {
        #     "numberOfTrees": 150,
        #     "subSampleSize": 512,
        #     "maxTreeDepth": 10,
        #     "extensionLevel": 1,
        #     "normalization": True
        # }
    # },
    "data": {
        # "version": {"id": "a74ae716-a86e-47f0-8a50-d8b21d6d7dd6"},
        "rows": {"type":"First","baseUnit": "Sample","value": 20000}, # or [{"from": "yyyy-mm-dd HH:MM:SS","to": "yyyy-mm-dd HH:MM:SS"}]
        "columns": subset+[target],
        "labelColumn": target,
        # "imputation": {"type": "LOCF","maxGapLength": 6},
        # "timeScale": {"baseUnit": "Hour","value": 1}
    }
}
detection_build_system_model_configuration

{'name': 'system_driven_subset',
 'useCase': {'id': 'af411cdd-7120-4622-870a-7e0790f2ae4a'},
 'experiment': {'id': 'edc99d69-cd1c-4e6c-9f99-461c406e1897'},
 'data': {'rows': {'type': 'First', 'baseUnit': 'Sample', 'value': 20000},
  'columns': ['M_Coupling Vibration',
   'M_casing_vib',
   'M_phase_current_C',
   'P_inlet_pressure',
   'M_phase_current_B',
   'BROKEN'],
  'labelColumn': 'BROKEN'}}

## 2.3 Model Building

In [21]:
# -----------------------------------------------------------------------------------------------------
detection_build_system_model = client.extended_detection.build_system_model(
    configuration = detection_build_system_model_configuration,
    outputs = [
        'id',
        'logs',
        'table',
    ],
    status_poll = print
 )
# -----------------------------------------------------------------------------------------------------
detection_build_system_model_id = detection_build_system_model.id
detection_build_system_model_logs = detection_build_system_model.logs
detection_build_system_model_table = detection_build_system_model.table

{'status': 'Running', 'createdAt': '2023-10-18T11:35:14.798Z'}
{'status': 'Running', 'progress': 27.5, 'CPU': 0.18, 'memory': 5333.0, 'createdAt': '2023-10-18T11:35:18.013Z'}
{'status': 'Finished', 'progress': 100.0, 'CPU': 0.18, 'memory': 5332.0, 'createdAt': '2023-10-18T11:35:19.310Z'}


In [22]:
detection_build_system_model_table['timestamp'] = pd.to_datetime(detection_build_system_model_table['timestamp'],format='%Y-%m-%dT%H:%M:%S.%fZ')

# 3. Results

## 3.1 Visual

In [ ]:
v_results = detection_build_system_model_table.copy()
v_data = tim_dataset[(tim_dataset[timestamp]>=v_results['timestamp'].min())&(tim_dataset[timestamp]<=v_results['timestamp'].max())].copy()
fig = splt.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=v_results['timestamp'], y=v_results['anomaly_code'], name='anomaly_code'), row=1, col=1)
fig.add_trace(go.Scatter(x=v_results['timestamp'], y=v_results['anomaly_indicator'], name='anomaly_indicator'), row=2, col=1)
fig.add_trace(go.Scatter(x=v_data[timestamp], y=v_data[target], name=target), row=1, col=1)
fig.add_hline(y=1, line_color="orange", row=2, col=1)
fig.update_layout(height=900, width=1200, title_text="Results visualization")
fig.show()    

# 3. Results

In [24]:
detect_from = str(detection_build_system_model_table['timestamp'].max())
detect_to = str(tim_dataset[timestamp].max())

detection_detect_configuration = {
    # "name": "My first anomaly detect job",
    "experiment": {"id": experiment_id},
    "data": {
        # "version": {"id": "a74ae716-a86e-47f0-8a50-d8b21d6d7dd6"},
        "rows": [{"from": detect_from,"to": detect_to}], #{"type":"Last","baseUnit": "Sample","value": 1} or [{"from": "yyyy-mm-dd HH:MM:SS","to": "yyyy-mm-dd HH:MM:SS"}]
        # "imputation": {"type": "LOCF","maxGapLength": 6}
    }
}
detection_detect_configuration

{'experiment': {'id': 'edc99d69-cd1c-4e6c-9f99-461c406e1897'},
 'data': {'rows': [{'from': '2018-04-14 21:19:00',
    'to': '2018-08-31 23:59:00'}]}}

In [25]:
detection_detect = client.extended_detection.detect(
    parent_job_id = detection_build_system_model_id,
    configuration = detection_detect_configuration,
    execute = True,
    wait_to_finish = True,
    outputs = [
        'id',
        'logs',
        'table',
    ],
    status_poll = print,
    tries_left = 300
 )

{'status': 'Running', 'createdAt': '2023-10-18T11:36:16.405Z'}
{'status': 'Running', 'createdAt': '2023-10-18T11:36:16.405Z'}
{'status': 'Running', 'progress': 9.5, 'CPU': 0.03, 'memory': 3677.0, 'createdAt': '2023-10-18T11:36:20.720Z'}
{'status': 'Running', 'progress': 9.5, 'CPU': 0.03, 'memory': 3677.0, 'createdAt': '2023-10-18T11:36:20.720Z'}
{'status': 'Running', 'progress': 95.0, 'CPU': 0.11, 'memory': 3687.0, 'createdAt': '2023-10-18T11:36:27.026Z'}
{'status': 'Running', 'progress': 95.0, 'CPU': 0.11, 'memory': 3687.0, 'createdAt': '2023-10-18T11:36:27.026Z'}
{'status': 'Running', 'progress': 95.0, 'CPU': 0.11, 'memory': 3687.0, 'createdAt': '2023-10-18T11:36:27.026Z'}
{'status': 'Running', 'progress': 95.0, 'CPU': 0.11, 'memory': 3687.0, 'createdAt': '2023-10-18T11:36:27.026Z'}
{'status': 'Running', 'progress': 95.0, 'CPU': 0.11, 'memory': 3687.0, 'createdAt': '2023-10-18T11:36:27.026Z'}
{'status': 'Running', 'progress': 95.0, 'CPU': 0.11, 'memory': 3687.0, 'createdAt': '2023-10

In [26]:
detection_detect_id = detection_detect.id
detection_detect_logs = detection_detect.logs
detection_detect_table = detection_detect.table

In [27]:
detection_detect_table['timestamp'] = pd.to_datetime(detection_detect_table['timestamp'],format='%Y-%m-%dT%H:%M:%S.%fZ')

In [ ]:
v_results = detection_detect_table.copy()
v_data = tim_dataset[(tim_dataset[timestamp]>=v_results['timestamp'].min())&(tim_dataset[timestamp]<=v_results['timestamp'].max())].copy()
fig = splt.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=v_results['timestamp'], y=v_results['anomaly_code'], name='anomaly_code'), row=1, col=1)
fig.add_trace(go.Scatter(x=v_results['timestamp'], y=v_results['anomaly_indicator'], name='anomaly_indicator'), row=2, col=1)
fig.add_trace(go.Scatter(x=v_data[timestamp], y=v_data[target], name=target), row=1, col=1)
fig.add_hline(y=1, line_color="orange", row=2, col=1)
fig.update_layout(height=900, width=1200, title_text="Results visualization")
fig.show()    